In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torch.utils.data import DataLoader, TensorDataset
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
class LSTMEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_layers=1):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True) # input dim (3) -> hidden dim (LSTM)
        self.fc = nn.Linear(hidden_dim, latent_dim) # hidden dim -> latent dim (z) (FC layer)

    def forward(self, x):
        # x: [B, T, C]
        h, _ = self.lstm(x)  # h: [B, T, H]
        h_last = h[:, -1, :]  # Last hidden state (sequence summary)
        z = self.fc(h_last)  # [B, latent_dim]
        return z

In [ ]:
class LSTMDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim, seq_len=10, num_layers=1):
        super(LSTMDecoder, self).__init__()
        self.seq_len = seq_len
        
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim) # latent dim -> hidden dim (FC layer)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True) #hidden dim -> hidden dim (LSTM)
        self.fc = nn.Linear(hidden_dim, output_dim) # hidden dim -> output dim (3) (FC layer)

    def forward(self, z):
        # z: [B, latent_dim]
        h0 = self.latent_to_hidden(z)  # latent vector [B, H] to hidden state
        h0_seq = h0.unsqueeze(1).repeat(1, self.seq_len, 1)  # Copy for the sequence length (T) [B, T, H]
        h_seq, _ = self.lstm(h0_seq) # Inference for 1 step (Make sequence)
        out = self.fc(h_seq)  # Recover for 3dim var with each time step output [B, T, output_dim]
        return out

In [ ]:
class USAD_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, seq_len=10):
        super(USAD_LSTM, self).__init__()
        self.encoder = LSTMEncoder(input_dim, hidden_dim, latent_dim)
        self.decoder1 = LSTMDecoder(latent_dim, hidden_dim, input_dim, seq_len)
        self.decoder2 = LSTMDecoder(latent_dim, hidden_dim, input_dim, seq_len)

    def forward(self, x):
        z = self.encoder(x)
        w1 = self.decoder1(z)
        w2 = self.decoder2(z)
        return w1, w2, z

In [ ]:
def data_norm(df,mode='train',scaler=''):
    """
    Normalize the data.
    df(dataframe) : Input
    return tmp(dataframe), scaler : normalized dataframe
    """
    columns = df.columns[1:]

    tmp = df.copy()
    if mode == 'train':
        # Normalize
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(tmp[columns])
    elif mode=='test':
        scaler = scaler
        scaled = scaler.transform(tmp[columns])
    # Insert the normalized value to the original frame
    tmp[columns] = scaled

    return tmp, scaler

In [ ]:
def inverse_norm(tensor, scaler):
    """
    tensor: [N, T, C] → numpy 배열로 변환 후 역정규화 수행
    """
    shape = tensor.shape
    data = tensor.reshape(-1, shape[-1]).cpu().numpy()
    inv = scaler.inverse_transform(data)
    return inv.reshape(shape)

In [ ]:
def create_sequences(df, seq_len=10):
    """
    날짜 포함된 시퀀스 생성 함수
    df: ['date', 'rn', 'vl', 'wl'] 포함된 DataFrame
    반환: (data_seq, date_seq)
        - data_seq: torch.Tensor [N, seq_len, 3]
        - date_seq: List[List[str]] [N, seq_len]
    """
    cols = df.columns[1:]  # 날짜 제외한 나머지 변수
    values = df[cols].values.astype(np.float32)
    dates = df['dates'].values  # 문자열 형태로 추출

    data_sequences = []
    date_sequences = []

    for i in range(len(df) - seq_len + 1):
        data_seq = values[i:i+seq_len]
        date_seq = dates[i:i+seq_len]
        data_sequences.extend(data_seq)
        date_sequences.extend(date_seq)

    return torch.tensor(np.stack(data_sequences)), date_sequences

In [ ]:
def create_combined_sequences(train_dfs, test_dfs, seq_len=120):
    """
    train_dfs(List): Input
    test_dfs(List): Input
    return train_seq(arr), val_seq(arr): train/val sliding window sequences tensor
    """
    
    train_seqs = []
    test_seqs = []
    train_dates = []
    test_dates = []

    for train, test in zip(train_dfs, test_dfs):
        train_seq, train_date = create_sequences(train, seq_len=seq_len)
        test_seq, test_date = create_sequences(test, seq_len=seq_len)
        train_seqs.append(train_seq)
        train_dates.append(train_date)
        test_seqs.append(test_seq)
        test_dates.append(test_date)
    
    return torch.cat(train_seqs, dim=0), torch.cat(test_seqs, dim=0), train_dates, test_dates

In [ ]:
def create_dataloaders(train_seq, val_seq, batch_size=64, shuffle=True):
    """
    Create the dataloader.
    train_seq(tensor)
    val_seq(tensor)
    return train_loader, val_loader
    """
    # Make TensorDataset 생성 (Same input and output)
    train_dataset = TensorDataset(train_seq, train_seq) # X, y
    val_dataset = TensorDataset(val_seq, train_seq) # X, y

    # Make DataLoader (Load X,y)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle) 
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

In [ ]:
def train_usad_lstm(model, train_loader, device='cuda', num_epochs=30, sav_path = '', alpha=0.5, beta=0.5, lr = 1e-4):
    """
    Train the model
    """
    # Initialize the model with device (CPU, GPU)
    model = model.to(device)

    # Use the ADAM optimizer
    optimizer1 = optim.Adam(list(model.encoder.parameters()) + list(model.decoder1.parameters()), lr=lr)
    optimizer2 = optim.Adam(list(model.encoder.parameters()) + list(model.decoder2.parameters()), lr=lr)
    # Loss function
    criterion = nn.MSELoss()

    best_loss = float('inf')  # 가장 작은 loss 추적용
    save_path = sav_path  # 저장할 폴더
    os.makedirs(save_path, exist_ok=True)

    loss1_list = []
    loss2_list = []

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss1, epoch_loss2 = 0.0, 0.0

        for x, _ in train_loader:
            x = x.to(device)

            # Phase 1
            # Initialize the gradient
            optimizer1.zero_grad()
            # Ouput w1 (_ : w2, z)
            w1, _, _ = model(x)
            # Calculate Loss function 1
            loss1 = alpha * criterion(w1, x)
            # Back-propagation
            loss1.backward()
            # Parameter update with the gradient
            optimizer1.step()
            epoch_loss1 += loss1.item()

            # Phase 2
            optimizer2.zero_grad()
            # Ouput w1 (_ : w1, z)
            _, w2, _ = model(x)
            # Calculate Loss function 2
            loss2 = beta * criterion(w2, x)
            # Back-propagation
            loss2.backward()
            # Parameter update with the gradient
            optimizer2.step()
            epoch_loss2 += loss2.item()

        avg_loss1 = epoch_loss1 / len(train_loader)
        avg_loss2 = epoch_loss2 / len(train_loader)

        loss1_list.append(avg_loss1)
        loss2_list.append(avg_loss2)

        print(f"[Epoch {epoch:03d}] Train Loss1: {epoch_loss1:.6f} | Train Loss2: {epoch_loss2:.6f}")

        # save weights file
        if avg_loss1 < best_loss:
            best_loss = avg_loss1
            torch.save(model.state_dict(), os.path.join(save_path, 'USAD_best.pth'))
            print(f"[Epoch {epoch}] 모델 저장됨: val_loss = {avg_loss1:.6f}")

    plt.figure(figsize=(8, 5))
    plt.plot(loss1_list, label='Loss1 (Encoder+Decoder1)')
    plt.plot(loss2_list, label='Loss2 (Encoder+Decoder2)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(sav_path, f'USAD_{seq_len}_train_loss_plot.png'))
    plt.close()
    print(f"[Saved] Training loss plot saved at {os.path.join(sav_path, f'USAD_{seq_len}_train_loss_plot.png')}")


In [ ]:
def test_usad_lstm(model, test_loader, device='cuda'):
    model.eval()
    model = model.to(device)
    criterion = nn.MSELoss()

    total_loss1, total_loss2 = 0.0, 0.0
    all_x, all_w2 = [], []

    with torch.no_grad():
        for x, _ in test_loader:
            x = x.to(device)
            w1, w2, _ = model(x)

            loss1 = criterion(w1, x).item()
            loss2 = criterion(w2, x).item()
            total_loss1 += loss1
            total_loss2 += loss2

            all_x.append(x.cpu())
            all_w2.append(w2.cpu())

    print(f"[Test] Loss1: {total_loss1:.6f} | Loss2: {total_loss2:.6f}")
    
    # 시계열 전체 보정 결과 반환
    all_x = torch.cat(all_x, dim=0)    # [N, 60, 3]
    all_w2 = torch.cat(all_w2, dim=0)  # [N, 60, 3]
    return all_x, all_w2


In [ ]:

def inverse_transform_column(scaler, normalized_tensor, column_index):
    """
    scaler: fitted MinMaxScaler
    normalized_tensor: torch.Tensor of shape [N, T]
    column_index: index in original data (0=rn, 1=vl, 2=wl)
    """
    # (1) Tensor → NumPy
    arr = normalized_tensor.detach().cpu().numpy()  # shape: [N, T]
    N, T = arr.shape

    # (2) Flatten to apply scaler: [N*T, 3]
    dummy = np.zeros((N*T, 3), dtype=np.float32)
    dummy[:, column_index] = arr.flatten()

    # (3) Inverse transform
    dummy_inv = scaler.inverse_transform(dummy)

    # (4) Extract only the column we care about
    result = dummy_inv[:, column_index].reshape(N, T)

    return result  # shape: [N, T]

In [ ]:
def compute_anomaly_scores(x_seq, w2_seq, mode='vl'):
    """
    x_seq, w2_seq: [N, T, 3]
    mode: 'vl' → 관로수위만 사용 / 'all' → 전체 변수 평균
    반환: [N] shape의 이상치 점수 벡터
    """
    # 관로수위 (index 2)만 비교
    errors = (x_seq - w2_seq) ** 2  # [N, T]
    scores = errors.mean(dim=1)  # 시퀀스별 평균 MSE → [N]
    return errors.numpy(), scores.numpy()

In [ ]:
def determine_threshold(scores, method='iqr', k=3):
    """
    scores: 이상치 점수 벡터 (numpy)
    method: 'iqr', 'mean_std', 'percentile'
    """
    if method == 'iqr':
        q1 = np.percentile(scores, 25)
        q3 = np.percentile(scores, 75)
        iqr = q3 - q1
        threshold = q3 + 1.5 * iqr

    elif method == 'mean_std':
        mean = np.mean(scores)
        std = np.std(scores)
        threshold = mean + k * std

    elif method == 'percentile':
        threshold = np.percentile(scores, 95)

    else:
        raise ValueError("지원하지 않는 방식입니다.")

    return threshold

In [ ]:
def plot_corrected_vs_original(original_seq, corrected_seq, input_seq, sav_path = '', sample_idx=0):
    """
    단일 시퀀스 비교 시각화 (60분간)
    """
    orig = original_seq[sample_idx]#.numpy()  # 관로 수위
    corr = corrected_seq[sample_idx]#.numpy()
    inputs = input_seq[sample_idx]
    if not os.path.isdir(sav_path):
        os.makedirs(sav_path)
    plt.figure(figsize=(12, 4))
    plt.plot(orig, label='Original (vl)', marker='o')
    plt.plot(corr, label='Corrected (w2)', marker='x')
    plt.plot(inputs, label='Input', marker='+')
    plt.title(f'Sample {sample_idx}: Sewer Level Original  vs Corrected')
    plt.xlabel('Time index (minute)')
    plt.ylabel('Sewer Level')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    plt.savefig(f'{sav_path}/{sample_idx}.png')

In [ ]:
def plot_anomalies_on_timeseries(original_seq, anomaly_flags, sav_path='', sample_idx=0):
    """
    Plot(scatter) the anomlay detection results on the graph
    original_seq : [N, T] or [N, T, 3]
    anomaly_flags : [N, T] (bool array)
    sample_idx(int)
    """
    vl = original_seq[sample_idx]  # shape: [T]
    flags = anomaly_flags[sample_idx]  # shape: [T], bool

    if not os.path.isdir(sav_path):
        os.makedirs(sav_path)

    plt.figure(figsize=(12, 4))
    plt.plot(vl, label='Sewer Level (vl)', color='blue')

    # Scatter the anomaly
    if flags.any():
        plt.scatter(np.where(flags)[0], vl[flags], color='red', label='Detected Anomaly', zorder=3)

    plt.title(f'Sample {sample_idx} - Detected {flags.sum()} Anomalies')
    plt.xlabel('Time index (minute)')
    plt.ylabel('Sewer Level')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    
    plt.savefig(f'{sav_path}/{sample_idx}_anomaly.png')
    plt.show()

In [ ]:
def summarize_anomaly_statistics(scores, anomaly_flags, original_values=None):
    """
    scores: numpy array [N, T] - 이상치 점수 (MSE 또는 RMSE)
    anomaly_flags: bool array [N, T] - 이상치 여부
    original_values: optional, numpy array [N, T] - 역정규화된 관측값, 결측치 개수 계산용
    """
    total_points = np.prod(scores.shape)
    num_anomalies = np.sum(anomaly_flags)
    num_normals = total_points - num_anomalies

    # 평균 오차
    mean_anomaly_score = np.mean(scores[anomaly_flags]) if num_anomalies > 0 else 0.0
    mean_normal_score = np.mean(scores[~anomaly_flags]) if num_normals > 0 else 0.0

    # 결측치 개수 계산 (입력된 경우에만)
    num_missing = 0
    if original_values is not None:
        num_missing = np.isnan(original_values).sum()

    print("📊 이상치 탐지 통계 요약")
    print(f"- 총 관측 시점 수: {total_points}")
    print(f"- 이상치 시점 수: {num_anomalies}")
    print(f"- 이상치 비율: {100.0 * num_anomalies / total_points:.2f}%")
    print(f"- 이상치 평균 오차: {mean_anomaly_score:.6f}")
    print(f"- 정상 평균 오차: {mean_normal_score:.6f}")
    if original_values is not None:
        print(f"- 결측치(Missing) 시점 수: {num_missing}")


In [ ]:
region = {0 : 'gwangjoo', 1 : 'changwon', 2 : 'pohang'}
fname = {0 : '2920010001045020', 1 : '4812110001018020', 2 : ''}
r_cd = 0

batch_size = 32
seq_len = 120
input_dim=3
hidden_dim=24
latent_dim=16

epochs = 100

data_dir = '../dataset/smart_rain/trainset/sewer/'
# 파일 경로 정렬
train_paths = sorted(glob.glob(os.path.join(data_dir, 'train*.csv')))
test_paths = sorted(glob.glob(os.path.join(data_dir, 'org*.csv')))

# 'flag' 열 제외하고 불러오기
usecols = ['dates', 'rn', 'vl', 'wl']  # 필요한 컬럼만 명시

train_dfs = [pd.read_csv(path, encoding='cp949', usecols=usecols) for path in train_paths]
test_dfs = [pd.read_csv(path, encoding='cp949', usecols=usecols) for path in test_paths]


In [ ]:
# 각 컬럼별 NaN 개수 누적용 (Series 초기화)
train_null = pd.Series(0, index=train_dfs[0].columns)
test_null = pd.Series(0, index=test_dfs[0].columns)

for train, test in zip(train_dfs, test_dfs):
    train_null += train.isnull().sum()
    test_null += test.isnull().sum()

print('Train set NaN 개수 (컬럼별):\n', train_null)
print('Test set NaN 개수 (컬럼별):\n', test_null)

# 전체 NaN 총합 출력
print(f"\nTrain set 전체 NaN 수: {train_null.sum()}")
print(f"Test set 전체 NaN 수: {test_null.sum()}")

In [ ]:
train_norm_dfs = []
test_norm_dfs = []
_, train_scaler = data_norm(pd.concat(train_dfs))

for train, test in zip(train_dfs, test_dfs):
    test.fillna(0.0, inplace=True)
    
    train_norm, _ = data_norm(train, scaler = train_scaler)
    test_norm, _ = data_norm(test, mode='test',scaler=train_scaler)
    
    test_norm['vl'] = test_norm['vl'].clip(lower= 0.0)
    
    train_norm_dfs.append(train_norm)
    test_norm_dfs.append(test_norm)

In [ ]:
print('▶ Train set')
tmp_orig = pd.concat(train_dfs)
tmp_norm = pd.concat(train_norm_dfs)

print('Rain (rn) - Original min/max:', tmp_orig['rn'].min(), tmp_orig['rn'].max())
print('Rain (rn) - Normalized min/max:', tmp_norm['rn'].min(), tmp_norm['rn'].max())

print('Sewer level (vl) - Original min/max:', tmp_orig['vl'].min(), tmp_orig['vl'].max())
print('Sewer level (vl) - Normalized min/max:', tmp_norm['vl'].min(), tmp_norm['vl'].max())

print('River level (wl) - Original min/max:', tmp_orig['wl'].min(), tmp_orig['wl'].max())
print('River level (wl) - Normalized min/max:', tmp_norm['wl'].min(), tmp_norm['wl'].max())

print('\n▶ Test set')
tmp_test_orig = pd.concat(test_dfs)
tmp_test_norm = pd.concat(test_norm_dfs)

print('Test set Nan count:\n', tmp_test_orig.isnull().sum())

print('Rain (rn) - Original min/max:', tmp_test_orig['rn'].min(), tmp_test_orig['rn'].max())
print('Rain (rn) - Normalized min/max:', tmp_test_norm['rn'].min(), tmp_test_norm['rn'].max())

print('Sewer level (vl) - Original min/max:', tmp_test_orig['vl'].min(), tmp_test_orig['vl'].max())
print('Sewer level (vl) - Normalized min/max:', tmp_test_norm['vl'].min(), tmp_test_norm['vl'].max())

print('River level (wl) - Original min/max:', tmp_test_orig['wl'].min(), tmp_test_orig['wl'].max())
print('River level (wl) - Normalized min/max:', tmp_test_norm['wl'].min(), tmp_test_norm['wl'].max())


In [ ]:
train_seq, val_seq, train_date, test_date = create_combined_sequences(train_norm_dfs, test_norm_dfs, seq_len=seq_len)

print("Train shape:", train_seq.shape)  # (N_train, 60, 3)
print("Val shape:", val_seq.shape)      # (N_val, 60, 3)

In [ ]:
train_date = np.stack(train_date)
test_date = np.stack(test_date)

In [ ]:
train_date.shape

In [ ]:
model = USAD_LSTM(input_dim=input_dim, hidden_dim=hidden_dim, latent_dim=latent_dim, seq_len=seq_len)
example = torch.randn((batch_size,seq_len, input_dim)) 
w1, w2, z = model(example)

print(w1.shape)  # torch.Size([32, 10, 3])
print(w2.shape)  # torch.Size([32, 10, 3])

In [ ]:
summary(model, input_size=(example.shape)) 

In [ ]:
# Create the data loader to load the dataset
train_loader, val_loader = create_dataloaders(train_seq, val_seq, batch_size=batch_size)

# Check the dataset shape
for x, y in train_loader:
    print(x.shape)  
    print(y.shape)  
    break


## Training

In [ ]:
# Training Start
train_usad_lstm(model, train_loader, device='cpu', num_epochs=epochs,sav_path=f'./sav/USAD_{seq_len}_rn_vl_wl/')


## Validation

In [ ]:
model.load_state_dict(torch.load(f'./sav/USAD_{seq_len}_rn_vl_wl/USAD_best.pth'))

In [ ]:

# Test
original_seq, corrected_seq = test_usad_lstm(model, val_loader, device='cpu')

# Select the correted sequence only sewer level"
original_vl =  original_seq[:, :, 1]  # [N, 60]
corrected_vl = corrected_seq[:, :, 1]  # [N, 60]

In [ ]:
# Caculate the anomaly scores
errors, scores = compute_anomaly_scores(original_vl, corrected_vl, mode='vl')  # 관로수위 기준

In [ ]:
# Determine the threshold
threshold = determine_threshold(scores, method='iqr')

plt.figure(figsize=(10,5))
plt.plot(scores)
plt.axhline(y=threshold, color='r', linestyle='--')
plt.title("Anomaly Scores Over Time (Each Sequence)")
plt.xlabel("Sequence Index")
plt.ylabel("Anomaly Score")
plt.grid(True)
plt.show()

In [ ]:
# 3. 이상치 탐지
anomaly_flags = errors > threshold

print("임계값:", threshold)
print("탐지된 이상치 수:", np.sum(anomaly_flags))

In [ ]:
sewer_lv = inverse_transform_column(train_scaler, original_vl, column_index=1) 
corrected_norm = inverse_transform_column(train_scaler, corrected_vl, column_index=1) 
corrected_norm[corrected_norm<0] = 0.0
input_seq = inverse_transform_column(train_scaler, val_seq[:,:,1], column_index=1) 

In [ ]:
corrected_norm = corrected_norm.round()
input_seq = input_seq.round()

In [ ]:
import random
sav_path = f'./imgs/USAD_{seq_len}/'
for i in range(50):
    n = random.randint(0,len(val_seq))
    plot_corrected_vs_original(sewer_lv, corrected_norm, input_seq, sav_path, sample_idx=n)
    plot_anomalies_on_timeseries(sewer_lv, anomaly_flags, sav_path, sample_idx=n)

In [ ]:

summarize_anomaly_statistics(errors, anomaly_flags, original_values=original_vl)

In [ ]:
def apply_anomaly_mask(original_seq, anomaly_flags, mask_value=np.nan, target_idx=1):
    """
    original_seq: numpy array [N, T, D] - 원본 시계열 데이터
    anomaly_flags: bool array [N, T] - 이상치 위치 (True: 이상치)
    mask_value: 마스킹할 값 (기본값: np.nan)
    target_idx: 마스킹할 변수 index (기본: 관로수위 = 1)
    return: 마스킹된 시계열 데이터 (복사본)
    """
    masked_seq = original_seq.copy()
    masked_seq[:, :, target_idx][anomaly_flags] = mask_value
    return masked_seq

In [ ]:
masked_seq = apply_anomaly_mask(original_seq.numpy(), anomaly_flags, mask_value=np.nan, target_idx=1)
